In [ ]:
!pip3 install -U spacy
!python -m spacy download en_core_web_sm
!pip3 install fuzzywuzzy
!pip3 install transformers
!cp /content/drive/MyDrive/fake-news-explainability/utils_fake_news.py .
%run utils_fake_news.py

     |████████████████████████████████| 5.9 MB 4.1 MB/s 
     |████████████████████████████████| 10.1 MB 49.8 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 451 kB 64.1 MB/s 
     |████████████████████████████████| 628 kB 59.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.6 MB 70 kB/s 
  Attempting uninstall: en-core-web-sm
    Foun

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz, process
import random
import itertools

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

def find_entities(text, type='PERSON'):
  doc = nlp(text)
  return [entity.text for entity in doc.ents if (entity.label_==type)\
          and (len(entity.text.split())>1)]

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/liar_valid.tsv', 
#                  delimiter='\t',
#                  header=None)
df = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/fake_news.csv')
df.columns = ['ID','label','statement','subject','speaker',
              'job_title','state','party','barely_true_count',
              'false_count','half_true_count','mostly_true_count',
              'pants_on_fire_count','context']

# Find names within statements, keep statements with names
df['entities'] = df['statement'].apply(find_entities)
df = df.loc[df.entities.apply(len)>0].reset_index(drop=True)

### Manual Encoding

In [ ]:
names = list(set(list(itertools.chain(*df['entities']))))
df_pol = pd.DataFrame({'name':names})

df_pol_raw = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/politicians_raw.csv')
df_pol['fuzzy'] = df_pol['name'].apply(lambda s: process.extractOne(s, df_pol_raw['Name']))
df_pol['match'] = df_pol['fuzzy'].apply(lambda x: x[0])
df_pol['score'] = df_pol['fuzzy'].apply(lambda x: x[1])
df_pol['index'] = df_pol['fuzzy'].apply(lambda x: x[2])
df_pol['recheck'] = df_pol['score'].apply(lambda x: 1 if x<95 else 0)
# df_names.to_csv('politicians.csv', index=False)

In [ ]:
df_pol = df_names

### Automated Processing

In [ ]:
# Reclassified data, keep only verified names
df_pol = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/politicians.csv',
                 encoding='latin-1')
df_pol = df_pol.loc[df_pol.recheck==0].reset_index(drop=True)

# Add party to names
df_raw = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/politicians_raw.csv')
df_raw.loc[df_raw.Political_party.str.contains('Republican'),'Political_party'] = 'Republican Party'
df_raw.loc[df_raw.Political_party.str.contains('Democratic'),'Political_party'] = 'Democratic Party'
df_pol['match'] = df_pol['index'].apply(lambda i: df_raw['Name'][i])
df_pol['party'] = df_pol['index'].apply(lambda i: df_raw['Political_party'][i])

# Fix names in entity list
entity_dict = {row['name']:row['match'] for (i,row) in df_pol.iterrows()}
df['entities_clean'] = df['entities'].apply(lambda lst: [entity_dict[name] for name in lst if name in entity_dict])

# Match party to name-statement pairs
party_dict = {row['match']:row['party'] for (i,row) in df_pol.iterrows()}
df['party'] = df['entities_clean'].apply(lambda lst: [party_dict[name] for name in lst if name in party_dict])

# Only keep statements that have names from 1 party
df = df.loc[df.party.apply(lambda l: len(set(l))==1)].reset_index(drop=True)
df['party'] = df['party'].apply(lambda l: l[0])
df = df[['id', 'label', 'statement', 'entities', 'entities_clean', 'party']]

# Collect list of names per party
df_party = df_pol.groupby('party').agg({'match':list}).reset_index().rename(columns={'match':'name'})
party_dict = {row['party']:row['name'] for (i,row) in df_party.iterrows()}

# Specify which party a replacement candidate should come from
df = df.loc[df.party.isin(['Democratic Party','Republican Party'])].reset_index(drop=True)
df['replacement_party'] = df['party'].apply(lambda x: 'Democratic Party' if x=='Republican Party' else 'Republican Party')

# Generate list of replacement candidates
df['replacement_names'] = df.apply(lambda row: random.sample(party_dict[row['replacement_party']], len(row['entities'])), axis=1)

# Replace
df['statement_new'] = df.apply(lambda row: replace_names(row['statement'],row['entities'],row['replacement_names']), axis=1)

In [ ]:
def replace_names(text, lst1, lst2):
    for (a,b) in zip(lst1,lst2):
        text = text.replace(a,b)
    return text

In [ ]:
df_orig = df[['id','label','statement']]
df_new  = df[['id','label','statement_new']].rename(columns={'statement_new':'statement'})

In [ ]:
# liar_encode = {'barely-true':1, 'false':1, 'pants-fire':1, 
#                'half-true':0, 'mostly-true':0, 'true':0}

# df_orig['label'] = df_orig['label'].apply(lambda x: liar_encode[x])
# df_new['label']  = df_new['label'].apply(lambda x: liar_encode[x])
df_new['label']  = df_new['label'].apply(lambda x: 0 if x==1 else 1)


In [ ]:
df_orig.to_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/fake_news_name_orig.csv', index=False)
df_new.to_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/fake_news_name_new.csv', index=False)

In [ ]:
torch.save(encode_dataframe(df_orig['statement'], df_orig['label']),
           '/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_name_orig.pt')
torch.save(encode_dataframe(df_new['statement'], df_new['label']),
           '/content/drive/MyDrive/fake-news-explainability/Data/Encoded/fake_news/evaluation/fake_news_name_new.pt')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
df_orig = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/liar_valid_name_orig.csv')
df_new = pd.read_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/liar_valid_name_new.csv')

In [ ]:
df_orig = df_orig.loc[df_orig.label==0].reset_index(drop=True)
df_new  = df_new.loc[df_new.label==1].reset_index(drop=True)

df_orig.to_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/liar_valid_name_orig_filtered.csv')
df_new.to_csv('/content/drive/MyDrive/fake-news-explainability/Data/Raw/liar_valid_name_new_filtered.csv')

In [ ]:
torch.save(encode_dataframe(df_orig['statement'], df_orig['label']),
           '/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_valid_name_orig_filtered.pt')
torch.save(encode_dataframe(df_new['statement'], df_new['label']),
           '/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_valid_name_new_filtered.pt')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
